In [1]:
import pandas as pd

# HUVEC


In [2]:
counts = pd.read_csv("./huvec/all_samples_gene_counts.txt", sep="\t", header=0)
print(counts.head)
print(counts.shape)
print(counts.columns)
new_columns = ['Gene'] + [name[9:] for name in counts.columns[1:]]
print(new_columns)
counts.columns = new_columns

<bound method NDFrame.head of               Gene  HUVEC_RNAd0_W1_R1  HUVEC_RNAd0_W0_R1  HUVEC_RNAd0_W2_R1  \
0          DDX11L1                  0                  0                  0   
1           WASH7P                112                246                132   
2        MIR6859-1                  0                  2                  4   
3      MIR1302-2HG                  0                  0                  0   
4        MIR1302-2                  0                  0                  0   
...            ...                ...                ...                ...   
50001          ND6              10230              29851              15385   
50002         TRNE                  0                  0                  0   
50003         CYTB              47313             118262              68154   
50004         TRNT                  0                  0                  0   
50005         TRNP                507                686                636   

       HUVEC_RNAd0_W3

In [3]:
# TPM normalizes using the length of the gene
# Allows us to more accurately compare abundance between genes (I think)
tpm = pd.read_csv("./huvec/all_samples_tpm_matrix.txt", sep="\t", header=0)
print(tpm.shape)
print(tpm.head)
print(tpm.columns)
dose_map = {"0.001" : "dA", "0.01": "dB", "0.1":"dC", "1.0": "dD", "2.0":"dE", "control": "d0"}
new_columns = ['Gene'] + [ dose_map[name.split("_")[1]] + "_" + name[0:2]+ "_" + name[-2:] for name in tpm.columns[1:] ]
print(new_columns)
tpm.columns = new_columns


(48021, 38)
<bound method NDFrame.head of        GeneID  W1_0.001_R1  W1_0.001_R2  W2_0.001_R1  W2_0.001_R2  \
0        A1BG     0.298326     0.673238     0.941353     1.074990   
1        A1CF     0.000000     0.000000     0.000000     0.000000   
2         A2M     0.030412     0.053326     0.056793     0.012807   
3       A2ML1     0.004552     0.006471     0.010968     0.002396   
4       A2MP1     0.000000     0.000000     0.000000     0.000000   
...       ...          ...          ...          ...          ...   
48016  ZYG11B     2.826560     3.174150     4.115840     4.209530   
48017     ZYX   239.398000   272.155000   365.450000   353.149000   
48018   ZYX_1   239.386000   272.140000   365.441000   353.141000   
48019   ZZEF1     2.533350     2.784970     3.693370     3.597940   
48020    ZZZ3     1.665650     1.949720     2.457780     2.410330   

       W3_0.001_R1  W3_0.001_R2  W1_0.01_R1  W1_0.01_R2  W2_0.01_R1  ...  \
0         1.076690     1.419650    0.583366    0.5152

In [8]:
metadata = pd.read_csv("./huvec/metadata_rna.csv", header=0)
print(metadata)

   SampleName ExposureRate_mGh  Week  Replicate  TotalExposure_mG
0    dA_W1_R1            0.001     1          1             0.168
1    dA_W1_R2            0.001     1          2             0.168
2    dA_W2_R1            0.001     2          1             0.336
3    dA_W2_R2            0.001     2          2             0.336
4    dA_W3_R1            0.001     3          1             0.504
5    dA_W3_R2            0.001     3          2             0.504
6    dB_W1_R1             0.01     1          1             1.680
7    dB_W1_R2             0.01     1          2             1.680
8    dB_W2_R1             0.01     2          1             3.360
9    dB_W2_R2             0.01     2          2             3.360
10   dB_W3_R1             0.01     3          1             5.040
11   dB_W3_R2             0.01     3          2             5.040
12   dC_W1_R1              0.1     1          1            16.800
13   dC_W1_R2              0.1     1          2            16.800
14   dC_W2

In [5]:
count_data_only = counts.drop(columns='Gene').T
drop_inds_count = count_data_only != 0
count_data_only = count_data_only.loc[:,(drop_inds_count).any(axis=0)]
print(count_data_only.shape)
genes_count = counts['Gene'].loc[(drop_inds_count).any(axis=0)]
print(genes_count.head)

(37, 29628)
<bound method NDFrame.head of 1              WASH7P
2           MIR6859-1
7        LOC124903816
8        LOC124900384
9           LOC729737
             ...     
50000             ND5
50001             ND6
50002            TRNE
50003            CYTB
50005            TRNP
Name: Gene, Length: 29628, dtype: object>


In [6]:
tpm_data_only = tpm.drop(columns="Gene").T
drop_inds_tpm = tpm_data_only != 0
tpm_data_only = tpm_data_only.loc[:,(drop_inds_tpm).any(axis=0)]
print(tpm_data_only.shape)
genes_tpm = tpm['Gene'].loc[(drop_inds_tpm).any(axis=0)]
print(genes_tpm.head)
print(len(set(genes_count).intersection(set(genes_tpm))))

(37, 37864)
<bound method NDFrame.head of 0           A1BG
1           A1CF
2            A2M
3          A2ML1
5        A3GALT2
          ...   
48016     ZYG11B
48017        ZYX
48018      ZYX_1
48019      ZZEF1
48020       ZZZ3
Name: Gene, Length: 37864, dtype: object>
28216


In [7]:
use_tpm = False # Must use count data because DESEQ2 is run on count matrix, so there are de genes that are not in the TPM matrix
genes = genes_tpm if use_tpm else genes_count 
data = tpm if use_tpm else counts
matrix = tpm_data_only if use_tpm else count_data_only 
dose = [f"{metadata.loc[metadata['SampleName']== name]['TotalExposure_mG'].values[0]}Gy" for name in data.columns[1:]]
dose_rate = [metadata.loc[metadata['SampleName']== name]['ExposureRate_mGh'].values[0] for name in data.columns[1:]]
print(set(dose))
print(set(dose_rate))
matrix.columns=genes
matrix['Dose'] = dose
matrix['Dose Rate'] = dose_rate

{'0.504Gy', '336.0Gy', '5.04Gy', '50.4Gy', '0.168Gy', '0.336Gy', '16.8Gy', '672.0Gy', '3.36Gy', '1008.0Gy', '0.0Gy', '504.0Gy', '33.6Gy', '168.0Gy', '1.68Gy'}
{'2.0', '1.0', 'Control', '0.1', '0.01', '0.001'}


In [8]:
print(matrix.columns)

Index(['WASH7P', 'MIR6859-1', 'LOC124903816', 'LOC124900384', 'LOC729737',
       'DDX11L17', 'WASH9P', 'MIR6859-2', 'LOC127239154', 'LOC124903815',
       ...
       'ND4L', 'ND4', 'TRNH', 'ND5', 'ND6', 'TRNE', 'CYTB', 'TRNP', 'Dose',
       'Dose Rate'],
      dtype='object', name='Gene', length=29630)


In [9]:
# Get DE genes for each dose 
dose_to_file_map = {"dA" : ["deseq2_0.168_W1vs0_by_week_results.csv", "deseq2_0.336_W2vs0_by_week_results.csv", "deseq2_0.504_W3vs0_by_week_results.csv"],
                         "dB": ["deseq2_1.68_W1vs0_by_week_results.csv","deseq2_3.36_W2vs0_by_week_results.csv", "deseq2_5.04_W3vs0_by_week_results.csv"],
                         "dC" : ["deseq2_16.8_W1vs0_by_week_results.csv","deseq2_33.6_W2vs0_by_week_results.csv", "deseq2_50.4_W3vs0_by_week_results.csv"],
                         "dD": ["deseq2_168_W1vs0_by_week_results.csv", "deseq2_336_W2vs0_by_week_results.csv", "deseq2_504_W3vs0_by_week_results.csv"],
                         "dE": ["deseq2_336_W1vs0_by_week_results.csv", "deseq2_672_W2vs0_by_week_results.csv", "deseq2_1008_W3vs0_by_week_results.csv"]
                         }
padj_value = 0.05
all_de_genes = {}
for dose, list_of_files in dose_to_file_map.items():
    print(dose)
    de_by_dose = []
    for file in list_of_files:
        de_df = pd.read_csv(f"./huvec/deseq2/{file}", header=0)
        de_genes = de_df.loc[de_df['padj']<=padj_value].iloc[:,0].values
        print(f"NUMBER OF DE GENES FOR WEEK {len(de_genes)}")
        print(f"NUMBER OF OVERLAP WITH PREVIOUS WEEK {len(set(de_by_dose).intersection(set(de_genes)))}")
        de_by_dose += list(de_genes)
    print(f"NUMBER OF DE GENES FOR DOSE {len(set(de_by_dose))}")
    all_de_genes[dose] = list(set(de_by_dose))
    
print(f"NUMBER OF DE GENES SHARED ACROSS ALL DOSES {len(set.intersection(*[set(i) for i in all_de_genes.values()]))}")

dA
NUMBER OF DE GENES FOR WEEK 2692
NUMBER OF OVERLAP WITH PREVIOUS WEEK 0


NUMBER OF DE GENES FOR WEEK 4133
NUMBER OF OVERLAP WITH PREVIOUS WEEK 1742
NUMBER OF DE GENES FOR WEEK 2404
NUMBER OF OVERLAP WITH PREVIOUS WEEK 1601
NUMBER OF DE GENES FOR DOSE 5886
dB
NUMBER OF DE GENES FOR WEEK 3701
NUMBER OF OVERLAP WITH PREVIOUS WEEK 0
NUMBER OF DE GENES FOR WEEK 4616
NUMBER OF OVERLAP WITH PREVIOUS WEEK 2324
NUMBER OF DE GENES FOR WEEK 3213
NUMBER OF OVERLAP WITH PREVIOUS WEEK 2228
NUMBER OF DE GENES FOR DOSE 6978
dC
NUMBER OF DE GENES FOR WEEK 3638
NUMBER OF OVERLAP WITH PREVIOUS WEEK 0
NUMBER OF DE GENES FOR WEEK 4676
NUMBER OF OVERLAP WITH PREVIOUS WEEK 2386
NUMBER OF DE GENES FOR WEEK 2202
NUMBER OF OVERLAP WITH PREVIOUS WEEK 1493
NUMBER OF DE GENES FOR DOSE 6637
dD
NUMBER OF DE GENES FOR WEEK 2658
NUMBER OF OVERLAP WITH PREVIOUS WEEK 0
NUMBER OF DE GENES FOR WEEK 2914
NUMBER OF OVERLAP WITH PREVIOUS WEEK 1246
NUMBER OF DE GENES FOR WEEK 2154
NUMBER OF OVERLAP WITH PREVIOUS WEEK 1332
NUMBER OF DE GENES FOR DOSE 5148
dE
NUMBER OF DE GENES FOR WEEK 2209
NUMBER 

# RPE1

In [2]:
tpm_rpe1 = pd.read_csv("./rpe1/rpe1_9week_study_all_tpm_data.csv")
print(tpm_rpe1)
de_df_rpe1 = pd.read_csv("./rpe1/rpe1_9week_study_diffexp_deseq_vs_control.txt", sep="\t")
print(de_df_rpe1)

         Gene Condition      TPM Dose Week
1        A1BG  dE_W3_R2  5.17818   dE   W3
2        A1BG  dC_W8_R2  3.51333   dC   W8
3        A1BG  d0_W3_R1  4.73971   d0   W3
4        A1BG  dA_W8_R1  4.66844   dA   W8
5        A1BG  dE_W5_R1  3.92758   dE   W5
...       ...       ...      ...  ...  ...
5234285  ZZZ3  dD_W9_R2  2.79447   dD   W9
5234286  ZZZ3  dC_W1_R2  3.21726   dC   W1
5234287  ZZZ3  dB_W4_R1  2.61975   dB   W4
5234288  ZZZ3  dC_W7_R1  2.84547   dC   W7
5234289  ZZZ3  dA_W7_R2  2.31725   dA   W7

[5234289 rows x 5 columns]
             Gene     baseMean  log2FoldChange     lfcSE      stat    pvalue  \
0            A1BG   295.811240        0.042450  0.234826  0.180771  0.856547   
1        A1BG-AS1    49.546145       -0.041171  0.502496 -0.081933  0.934700   
2            A1CF     0.000000             NaN       NaN       NaN       NaN   
3             A2M     0.000000             NaN       NaN       NaN       NaN   
4         A2M-AS1     0.000000             NaN       NaN

In [3]:
metadata = pd.read_csv("./rpe1/metadata.csv", header=0, sep=',')
print(metadata)

   Week Dose  Exposure
0    W1   dC    16.800
1    W1   dA     0.168
2    W1   dD   168.000
3    W1   dE   336.000
4    W1   dB     1.680
5    W2   dC    33.600
6    W2   dA     0.336
7    W2   dD   336.000
8    W2   dE   672.000
9    W2   dB     3.360
10   W3   dC    50.400
11   W3   dA     0.504
12   W3   dD   504.000
13   W3   dE  1008.000
14   W3   dB     5.040
15   W4   dC    67.200
16   W4   dA     0.672
17   W4   dD   672.000
18   W4   dE  1344.000
19   W4   dB     6.720
20   W5   dC    84.000
21   W5   dA     0.840
22   W5   dD   840.000
23   W5   dE  1680.000
24   W5   dB     8.400
25   W6   dC   100.800
26   W6   dA     1.008
27   W6   dD  1008.000
28   W6   dE  2016.000
29   W6   dB    10.080
30   W7   dC   117.600
31   W7   dA     1.176
32   W7   dD  1176.000
33   W7   dE  2352.000
34   W7   dB    11.760
35   W8   dC   134.400
36   W8   dA     1.344
37   W8   dD  1344.000
38   W8   dE  2688.000
39   W8   dB    13.440
40   W9   dC   151.200
41   W9   dA     1.512
42   W9   d

In [10]:
dose_rates = list(set(tpm_rpe1["Dose"]))
padj_value = 0.05
for d in dose_rates:
    print(d)
    if d == "d0" or d=='Day':
        continue
    tpm_subset = tpm_rpe1[tpm_rpe1["Dose"].isin([d, "d0"])]
    de_subset = de_df_rpe1[de_df_rpe1["Dose"]==d]
    de_genes = de_subset.loc[de_subset['padj']<=padj_value].iloc[:,0].values
    matrix_by_dose_de = tpm_subset[tpm_subset["Gene"].isin(de_genes)]
    matrix_by_dose_de = [x for _,x in matrix_by_dose_de.groupby('Gene')]
    gene_columns = []
    for x in matrix_by_dose_de:
        y = pd.DataFrame(data=x['TPM'].values)
        y.columns=[x['Gene'].values[0]]
        gene_columns.append(y)
    gene_columns.append(pd.DataFrame(data=x['Condition'].values))
    matrix_by_dose_de = pd.concat(gene_columns, axis=1, join='inner')
    # print(matrix_by_dose_de.iloc[:,-1])
    # Add dose column
    dose = [metadata.loc[metadata['Week']== name[3:5]].loc[metadata['Dose']== name[0:2]]['Exposure'].values[0] if name[0:2] != "d0" else 0 for name in matrix_by_dose_de.iloc[:,-1]]
    matrix_by_dose_de.iloc[:,-1] = dose
    matrix_by_dose_de.rename(columns={0:'radiation'}, inplace=True)
    # print(matrix_by_dose_de)
    matrix_by_dose_de.to_csv(f"./rpe1/cd_tpm_matrix_{d}.csv", header=True, index=False)

    

dE


/tmp/ipykernel_17474/2337543050.py:22: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  matrix_by_dose_de.iloc[:,-1] = dose


dC


/tmp/ipykernel_17474/2337543050.py:22: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  matrix_by_dose_de.iloc[:,-1] = dose


dA


...

Day
dD


/tmp/ipykernel_17474/2337543050.py:22: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  matrix_by_dose_de.iloc[:,-1] = dose


dB


/tmp/ipykernel_17474/2337543050.py:22: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  matrix_by_dose_de.iloc[:,-1] = dose


d0


In [34]:
list(set(de_df_rpe1["Dose"]))

['dB', 'dD', 'dA', 'dC', 'dE']

# RPE1 campaign 2

In [2]:
tpm_rpe1_exp2 = pd.read_csv("./rpe1_experiment2/rpe1_9week_study_experiment2_all_tpm.tsv", sep="\t")
print(tpm_rpe1_exp2)
de_df_rpe1_exp2 = pd.read_csv("./rpe1_experiment2/rpe1_9week_study_experiment2_diffexp_deseq_vs_control_all_dG_W2_adjust.txt", sep="\t")
print(de_df_rpe1_exp2)

         Gene    dF_W8_R1    dJ_W8_R1    dH_W8_R2    dJ_W9_R1    dH_W9_R2  \
0        A1BG    4.374090    4.198690    3.381290    3.952830    3.386160   
1        A1CF    0.006639    0.000000    0.000000    0.002389    0.000000   
2         A2M    0.000000    0.000000    0.000000    0.004244    0.004621   
3       A2ML1    0.004414    0.002981    0.002726    0.006352    0.006916   
4       A2MP1    0.000000    0.000000    0.000000    0.000000    0.000000   
...       ...         ...         ...         ...         ...         ...   
48016  ZYG11B    3.968280    3.456620    3.575250    3.428520    3.502670   
48017     ZYX  304.679000  309.062000  351.233000  347.555000  329.011000   
48018   ZYX_1  304.657000  309.072000  351.233000  347.555000  329.011000   
48019   ZZEF1    2.199460    1.786760    1.873860    1.874630    2.058750   
48020    ZZZ3    3.658560    2.645900    3.389560    2.834250    3.625010   

         dF_W9_R1    dG_W8_R1    d0_W8_R1    dI_W8_R2  ...    dI_W4_R1  \
0

In [ ]:
import numpy as np
#dose_rates = list(set(de_df_rpe1_exp2["Dose"]))
dose_rates = ["d0", "dF", "dG", "dH", "dI", "dJ"]
dose_rate_val = [0, 0.004, 0.04, 0.4, 4, 8]
print(dose_rates)
padj_value = 0.05
exp_ids = list(set(tpm_rpe1_exp2.columns))
print(exp_ids)
for d,val in zip(dose_rates, dose_rate_val):
    print(d, val)
    tpm_subset = tpm_rpe1_exp2[['Gene']+[id for id in exp_ids if (id[0:2] == d)]]
    if val != 0:
        de_subset = de_df_rpe1_exp2[de_df_rpe1_exp2["Dose"]==d]
        de_genes = de_subset.loc[de_subset['padj']<=padj_value].iloc[:,0].values
        matrix_by_dose_de = tpm_subset[tpm_subset["Gene"].isin(de_genes)]
    else:
        matrix_by_dose_de = tpm_subset
    matrix_by_dose_de = matrix_by_dose_de.set_index('Gene').T.rename_axis('Exp ID').rename_axis(None,axis=1)
    calculated_dose = [ val*168*int(week_num[4]) for week_num,_ in matrix_by_dose_de.iterrows()]
    matrix_by_dose_de['radiation'] = calculated_dose 
    matrix_by_dose_de['week'] = [int(week_num[4]) for week_num,_ in matrix_by_dose_de.iterrows()] 
    print(matrix_by_dose_de.shape)
    print(calculated_dose)
    matrix_by_dose_de.to_csv(f"./rpe1_experiment2/cd_tpm_matrix__fixed_{d}.csv", header=True, index=False)

    

['d0', 'dF', 'dG', 'dH', 'dI', 'dJ']
['dH_W5_R2', 'dG_W1_R1', 'Day_0', 'd0_W1_R1', 'dI_W8_R1', 'dJ_W6_R1', 'dG_W6_R2', 'dG_W4_R1', 'dI_W7_R1', 'd0_W5_R1', 'dG_W7_R2', 'd0_W9_R2', 'dJ_W2_R1', 'dI_W8_R2', 'dI_W3_R2', 'dJ_W4_R1', 'dH_W4_R2', 'd0_W6_R2', 'dG_W4_R2', 'dI_W2_R2', 'dI_W9_R1', 'dF_W3_R2', 'dJ_W9_R2', 'd0_W3_R1', 'dI_W6_R1', 'd0_W5_R2', 'd0_W3_R2', 'dJ_W3_R1', 'dF_W3_R1', 'dH_W3_R1', 'dI_W4_R1', 'dH_W4_R1', 'Gene', 'dI_W4_R2', 'dH_W9_R1', 'dH_W9_R2', 'd0_W8_R1', 'dF_W1_R2', 'dG_W6_R1', 'dG_W2_R1', 'dF_W8_R1', 'd0_W9_R1', 'dF_W7_R2', 'dJ_W6_R2', 'dJ_W1_R2', 'd0_W7_R1', 'dH_W8_R1', 'dH_W2_R2', 'd0_W4_R1', 'd0_W4_R2', 'dI_W2_R1', 'dF_W6_R2', 'd0_W7_R2', 'dF_W8_R2', 'dF_W2_R1', 'dH_W8_R2', 'd0_W6_R1', 'dG_W9_R1', 'dI_W6_R2', 'dG_W7_R1', 'dI_W5_R2', 'dJ_W3_R2', 'dH_W6_R1', 'dI_W3_R1', 'dH_W2_R1', 'dF_W9_R1', 'dH_W1_R2', 'dJ_W4_R2', 'dH_W1_R1', 'dF_W2_R2', 'd0_W8_R2', 'dG_W2_R2', 'dJ_W5_R2', 'd0_W2_R2', 'd0_W1_R2', 'dJ_W2_R2', 'dJ_W7_R2', 'dG_W3_R2', 'dG_W5_R2', 'dG_W8_R1', 'dJ_W8_R2

In [3]:
import numpy as np
dose_rates = ["d0", "dF", "dG", "dH", "dI", "dJ"]
dose_rate_val = [0, 0.004, 0.04, 0.4, 4, 8]
padj_value = 0.05
exp_ids = list(set(tpm_rpe1_exp2.columns))
print(exp_ids)
dfs = []
genes = set()
for d,val in zip(dose_rates, dose_rate_val):
    print(d, val)
    tpm_subset = tpm_rpe1_exp2[['Gene']+[id for id in exp_ids if (id[0:2] == d)]]
    if val != 0:
        de_subset = de_df_rpe1_exp2[de_df_rpe1_exp2["Dose"]==d]
        de_genes = de_subset.loc[de_subset['padj']<=padj_value].iloc[:,0].values
        genes = genes.union(de_genes)
    matrix_by_dose = tpm_subset.set_index('Gene').T.rename_axis('Exp ID').rename_axis(None,axis=1)
    calculated_dose = [ val*168*int(week_num[4]) for week_num,_ in matrix_by_dose.iterrows()]
    matrix_by_dose['week'] = [int(week_num[4]) for week_num,_ in matrix_by_dose.iterrows()] 
    matrix_by_dose['dose_rate'] = val* np.ones(matrix_by_dose.shape[0])
    dfs.append(matrix_by_dose)
    # print(matrix_by_dose.shape)
    # print(calculated_dose)
genes.add('week')
genes.add('dose_rate')
print(len(genes), genes)

df_combined = pd.concat(dfs)
print(df_combined.columns)

df_combined = df_combined[[g for g in genes if g in df_combined.columns]]
"radiation" in df_combined.columns
#df_combined.to_csv(f"./rpe1_experiment2/cd_tpm_matrix_combined_dose_rate.csv", header=True, index=False)


['dJ_W9_R1', 'dF_W2_R1', 'd0_W8_R1', 'dF_W8_R1', 'dI_W6_R1', 'dJ_W4_R1', 'dI_W1_R1', 'dG_W4_R2', 'd0_W1_R2', 'dH_W2_R2', 'd0_W2_R1', 'dF_W2_R2', 'dI_W7_R1', 'd0_W3_R2', 'dH_W5_R1', 'dF_W3_R1', 'dI_W8_R1', 'dH_W8_R1', 'dI_W8_R2', 'dH_W5_R2', 'dF_W5_R2', 'dF_W7_R1', 'dG_W8_R2', 'dF_W5_R1', 'dJ_W9_R2', 'd0_W8_R2', 'dG_W2_R1', 'dI_W4_R1', 'dJ_W1_R2', 'dJ_W1_R1', 'dJ_W7_R2', 'dJ_W5_R1', 'dH_W1_R1', 'dI_W2_R1', 'd0_W9_R1', 'dF_W9_R2', 'dJ_W3_R2', 'dH_W6_R2', 'dH_W1_R2', 'dI_W9_R2', 'Gene', 'dG_W6_R2', 'dH_W7_R1', 'd0_W6_R1', 'dI_W4_R2', 'dH_W4_R2', 'dG_W4_R1', 'dH_W8_R2', 'dI_W9_R1', 'dG_W1_R2', 'd0_W9_R2', 'd0_W7_R2', 'dI_W6_R2', 'dG_W7_R1', 'dG_W9_R2', 'd0_W4_R1', 'd0_W3_R1', 'dG_W3_R1', 'dJ_W4_R2', 'dF_W8_R2', 'dH_W4_R1', 'dF_W3_R2', 'dJ_W8_R1', 'dI_W2_R2', 'd0_W7_R1', 'dI_W3_R1', 'dH_W3_R2', 'dF_W6_R2', 'dG_W6_R1', 'dI_W5_R2', 'dG_W5_R2', 'dH_W9_R2', 'd0_W2_R2', 'dG_W8_R1', 'd0_W5_R1', 'dH_W3_R1', 'dG_W3_R2', 'd0_W4_R2', 'dH_W6_R1', 'dJ_W2_R2', 'dG_W1_R1', 'dJ_W8_R2', 'dI_W3_R2', 'dH_W2_

False

# CD Matrices

In [10]:
# Now save files for each dose (should be 13 samples each for 3 weeks x 2 replicates + 7 controls)
for dose in dose_to_file_map.keys():
    print(dose)
    matrix_by_dose =  matrix.filter(regex=f"d0+|{dose}+", axis=0)
    print(matrix_by_dose.shape)
    print(set(matrix_by_dose.columns.values).intersection(all_de_genes[dose]))
    print(all_de_genes[dose])
    matrix_by_dose_de = matrix_by_dose[all_de_genes[dose]]
    matrix_by_dose_de['radiation'] = [float(dose_str[0:len(dose_str)-2]) for dose_str in matrix_by_dose["Dose"]]
    print(matrix_by_dose_de.shape)
    matrix_by_dose_de.to_csv(f"./huvec/cd_matrix_{dose}.csv", header=True, index=False)
    de_genes_out = pd.DataFrame(data = all_de_genes[dose])
    de_genes_out.to_csv(f"./huvec/cd_genes_{dose}.csv", header=True, index=False)

    

dA
(13, 29630)
{'RPL23AP53', 'HSPH1', 'DBT', 'CCNA1', 'STAMBPL1', 'LBH', 'CLEC11A', 'HNRNPA3', 'SULF1', 'CASP1', 'B4GALT1', 'SRM', 'TGFB1', 'LRRC20', 'POLL', 'MAGED1', 'ERCC6L', 'CNOT9', 'MGME1', 'PDLIM1', 'AGPS', 'P3H4', 'BTG1', 'R3HCC1', 'SNX7', 'PUS3', 'TNFRSF21', 'FN3KRP', 'DOCK4', 'PGF', 'PIK3C2A', 'FLI1', 'MMP16', 'RIC8A', 'RPS14', 'LMBRD1', 'C2CD5', 'RFC2', 'HPRT1', 'PAK4', 'DYNC2I2', 'MAN1A1', 'SRSF2', 'ZSWIM3', 'GOLPH3', 'PRXL2C', 'MPDZ', 'ORC6', 'ALG3', 'CHAF1A', 'TRAM2', 'PPP2R2A', 'LRRC17', 'CREB3L1', 'NUP37', 'FLNB', 'SORBS2', 'ZCCHC17', 'PCTP', 'PARP10', 'NOP53', 'NGLY1', 'TMF1', 'MYH10', 'LRRC32', 'ARHGEF18', 'PIMREG', 'CCIN', 'ADARB1', 'KIF3A', 'MPP2', 'RELB', 'HLX', 'CRELD2', 'PHACTR4', 'ISOC1', 'CDK19', 'PRPS1', 'SEC24D', 'MMP24OS', 'INSYN2B', 'PTPN18', 'CUBN', 'TYRO3', 'MAP2K6', 'AHCTF1', 'TRA2B', 'S1PR1', 'SPOP', 'KIF13A', 'MVK', 'SLC16A5', 'ZNF862', 'SLC26A4', 'HP1BP3', 'MAP1B', 'IPO5', 'EXOSC1', 'DLAT', 'TKT', 'SNRPB', 'LSM3', 'TSPAN12', 'COQ2', 'NOL3', 'RPGR', 'C

/tmp/ipykernel_231533/2770062464.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matrix_by_dose_de['radiation'] = [float(dose_str[0:len(dose_str)-2]) for dose_str in matrix_by_dose["Dose"]]
/tmp/ipykernel_231533/2770062464.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matrix_by_dose_de['radiation'] = [float(dose_str[0:len(dose_str)-2]) for dose_str in matrix_by_dose["Dose"]]
/tmp/ipykernel_231533/2770062464.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

dD
(13, 29630)
{'HSPH1', 'DBT', 'CCNA1', 'STAMBPL1', 'CLEC11A', 'ZFHX3', 'SULF1', 'CSTF1', 'CASP1', 'SRM', 'SHPRH', 'ILRUN', 'TGFB1', 'MAGED1', 'OGFR', 'ERCC6L', 'CNOT9', 'MGME1', 'PDLIM1', 'P3H4', 'BTG1', 'CDKN2D', 'SNX7', 'SCOC', 'TNFRSF21', 'DOCK4', 'PGF', 'PIK3C2A', 'FLI1', 'LGALS8', 'MMP16', 'RPS14', 'ARHGAP4', 'LMBRD1', 'RFC2', 'HPRT1', 'PAK4', 'DYNC2I2', 'MAN1A1', 'VPS41', 'ATP2A3', 'SRSF2', 'C2CD2L', 'MPDZ', 'PRXL2C', 'ALG3', 'TRAM2', 'BCAS4', 'CHAF1A', 'PPP2R2A', 'UCP2', 'LRRC17', 'CREB3L1', 'NUP37', 'FLNB', 'SORBS2', 'TOMM22', 'ZCCHC17', 'PARP10', 'NOP53', 'NGLY1', 'TMF1', 'MYH10', 'ARHGEF18', 'LRRC32', 'PIMREG', 'BCL7C', 'RPL35', 'ADARB1', 'MPP2', 'MEA1', 'HLX', 'CRELD2', 'PHACTR4', 'ISOC1', 'CDK19', 'SEC24D', 'RNF141', 'INSYN2B', 'CUBN', 'TYRO3', 'TM9SF4', 'MAP2K6', 'TRA2B', 'S1PR1', 'SPOP', 'KIF13A', 'MVK', 'REV3L', 'SLC16A5', 'ZNF862', 'XPA', 'CDK9', 'MAP1B', 'DPP7', 'EXOSC1', 'DLAT', 'TKT', 'EDA2R', 'CUX1', 'SNRPB', 'LSM3', 'TSPAN12', 'COQ2', 'NOL3', 'RPGR', 'ARF1', 'COA

/tmp/ipykernel_231533/2770062464.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matrix_by_dose_de['radiation'] = [float(dose_str[0:len(dose_str)-2]) for dose_str in matrix_by_dose["Dose"]]
/tmp/ipykernel_231533/2770062464.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matrix_by_dose_de['radiation'] = [float(dose_str[0:len(dose_str)-2]) for dose_str in matrix_by_dose["Dose"]]


# GENELink data files

In [ ]:
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
path = "./huvec/GENELink_data_files"
if not os.path.exists(path):
    os.makedirs(path)
# Create ExpressionData_dose.csv files for huvec data
for dose in dose_to_file_map.keys():
    print(dose)
    matrix_by_dose =  matrix.filter(regex=f"d0+|{dose}+", axis=0)
    print(matrix_by_dose.shape)
    print(set(matrix_by_dose.columns.values).intersection(all_de_genes[dose]))
    print(all_de_genes[dose])
    matrix_by_dose_de = matrix_by_dose[all_de_genes[dose]]
    #Normalize matrix 
    data = MinMaxScaler().fit_transform(matrix_by_dose_de)
    matrix_by_dose_de_norm = pd.DataFrame(data=data)
    matrix_by_dose_de_norm.columns = matrix_by_dose_de.columns
    
    matrix_by_dose_de_norm['radiation'] = [float(dose_str[0:len(dose_str)-2]) for dose_str in matrix_by_dose["Dose"]]
    print(matrix_by_dose_de_norm.shape)
    matrix_by_dose_de_norm.to_csv(f"{path}/GeneExpression_{dose}.csv", header=True, index=False)




dA
(13, 29630)
{'ABHD17A', 'HEXIM1', 'TOM1L2', 'NSUN2', 'JARID2', 'TMEM217', 'TMEM163', 'JMJD1C', 'RBBP8', 'GNA14', 'ARHGAP27_1', 'FAS', 'SHISA2', 'DNAJC3', 'COL6A1', 'HDGFL3', 'ACTB', 'SLC29A2', 'SCAF1', 'SLC25A33', 'DDX39A', 'ATAD5', 'LSM5', 'S100A6', 'EIF2S1', 'XPO4', 'HACD3', 'REL', 'WWC2', 'OLFML2A', 'ATP2A2', 'ACTR3', 'GANAB', 'DCPS', 'TNS1', 'PDE2A', 'TMEM171', 'CDK4', 'LETM2', 'INTS1', 'NUDT19', 'CUEDC1', 'CBX6', 'USP19', 'VSIG10', 'LOC124902029', 'ENO1', 'MSH6', 'ITGA2', 'GSTK1', 'COASY', 'PYGO1', 'HNRNPUL1', 'MRE11', 'ILF2', 'RPL9', 'SPR', 'LMCD1', 'SACS', 'CAB39', 'TIAM2', 'ZNRD2', 'CENPX', 'CYRIA', 'RCSD1', 'STMP1', 'CIB2', 'DNASE2', 'CSTF2T', 'SELENOI', 'SCAI', 'ELK4', 'ACSL4', 'UBFD1', 'SNRPF', 'MAMDC2', 'TOPBP1', 'SLFN11', 'PPP3CB', 'MON1A', 'UBE4B', 'KIAA0040', 'SLC50A1', 'PLK4', 'AKAP12', 'AGFG2', 'ERCC6L', 'TMEM63B', 'ZMAT3', 'LIMD1', 'IL18', 'RELB', 'NCSTN', 'RALGDS', 'TAGLN2', 'APPBP2', 'STS', 'SIRPA', 'IGF1R', 'MANBA', 'ZFYVE1', 'OAT', 'CHCHD7', 'TOM1L1', 'ANXA1', 

In [2]:
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
path = "./rpe1_experiment2/GENELink_data_files"
if not os.path.exists(path):
    os.makedirs(path)
# Create ExpressionData_dose.csv files for rpe1 data
for dose in ["F", "G", "H", "I", "J"]:
    # print(dose)
    # matrix_by_dose =  matrix.filter(regex=f"d0+|{dose}+", axis=0)
    # print(matrix_by_dose.shape)
    # print(set(matrix_by_dose.columns.values).intersection(all_de_genes[dose]))
    # print(all_de_genes[dose])
    # matrix_by_dose_de = matrix_by_dose[all_de_genes[dose]]
    matrix_by_dose_de = pd.read_csv(f"./rpe1_experiment2/cd_tpm_matrix_d{dose}.csv", header=0)
    #Normalize matrix 
    data = MinMaxScaler().fit_transform(matrix_by_dose_de)
    matrix_by_dose_de_norm = pd.DataFrame(data=data)
    matrix_by_dose_de_norm.columns = matrix_by_dose_de.columns
    
    # matrix_by_dose_de_norm['radiation'] = [float(dose_str[0:len(dose_str)-2]) for dose_str in matrix_by_dose["Dose"]]
    # print(matrix_by_dose_de_norm.shape)
    matrix_by_dose_de_norm.to_csv(f"{path}/GeneExpression_{dose}_key_genes.csv", header=True, index=False)




### Compare DE genes with ChatGPT genes

In [37]:
genes_gpt = pd.read_csv("./prior_knowledge/radiation_response_genes_132_chatgpt.csv")["Gene"].values
genes_curated = pd.read_csv("./prior_knowledge/radiation_genes.txt", sep=" ", header=None).iloc[:,0].values
in_de_set = []
for g in genes_gpt:
    flag=False
    for dose,de_genes in all_de_genes.items():
        flag = (g in de_genes) or flag
    if flag:
        in_de_set.append(g)
print(len(genes_gpt), len(in_de_set))
print(in_de_set)

132 89
['FDXR', 'DDB2', 'GADD45A', 'CDKN1A', 'PCNA', 'BBC3', 'BAX', 'CCNG1', 'TNFRSF10B', 'SESN1', 'MDM2', 'XPC', 'POLH', 'PLK2', 'FAS', 'GDF15', 'TIGAR', 'DRAM1', 'TNFSF4', 'AEN', 'ZMAT3', 'BCL2L11', 'BTG2', 'GADD45B', 'ATM', 'MAPK14', 'H2AX', 'RAD51', 'LIG4', 'XRCC4', 'RNF168', 'RNF8', 'BRCA1', 'BRCA2', 'ABRAXAS1', 'BABAM2', 'RAD51AP1', 'RHNO1', 'TP53', 'USP28', 'UIMC1', 'KAT5', 'SIRT1', 'IFI16', 'CLOCK', 'MEN1', 'STK11', 'PRKAA1', 'TELO2', 'TOPBP1', 'TICRR', 'DCUN1D3', 'BLM', 'BRAT1', 'FIGNL1', 'PAXIP1', 'IKBIP', 'RHOB', 'SMPD1', 'TANK', 'TNKS1BP1', 'VWCE', 'PLK3', 'SERPINE1', 'NABP1', 'NABP2', 'RFWD3', 'PHLDA3', 'TNFRSF10A', 'EDA2R', 'PTCHD4', 'PAPPA', 'MKI67', 'FAM111B', 'ZNF367', 'LOX', 'LOXL1', 'LOXL2', 'LOXL3', 'LOXL4', 'COX7C', 'QSOX1', 'GPX1', 'IL7R', 'IRS2', 'TUBA1A', 'MYADM', 'DUSP5', 'HMGB2']


In [36]:
"CREB1" in all_de_genes["dA"]

False